In [1]:
import numpy as np
import math

##Absorbing Markov Chains
An absorbing MC is one that is impossible to leave, i.e. in the transition matrix there exists two states where the probability is 1. In the absorbing MC a state which does not have probability of 1 is called a transient state.

In the *Drunkard's Walk* 0,1,2,3,4 are corners. At 0 and 4 are the drunkard's bar and home respectively. If he arrives at either he will stay there, i.e P=1.

![Drunkard diagram](/files/Drunkard_walk.svg)

| 0 | 1 | 2 | 3 | 4 |
| --|:--:|:--:|:--:| --:|
| 1 | 0 | 0 | 0 | 0 |
|1/2| 0 |1/2| 0 | 0 |
| 0 |1/2| 0 |1/2| 0 |
| 0 | 0 |1/2| 0 |1/2|
| 0 | 0 | 0 | 0 | 1 |


We have to render this matrix in canonical form of:

|Transition|Absorbing|
|----------|--------:|
|**Q**|**R**|
|**0**|**I**|

Where Q is $tr$ x $tr$ matrix (*tr* are the transition states i.e 1,2,3 -> 3x3); R is $tr$ x $abs$ matrix (*abs* is the absorbtion matrix -> 3x2) 0 is an *tr* x *abs* 0-matrix and I is *abs* x *abs* identity matrix.

1. Q = rows(1,2,3) x col(1,2,3)
2. R = rows(1,2,3) x col(0,4)
3. 0 = rows(0,4) x col(1,2,3)
4. I = rows(0,4) x col(0,4)

Resulting in 

| 0 | 1 | 2 | 3 | 4 |
| --|:--:|:--:|:--:| --:|
| 0 |1/2| 0 |1/2| 0 |
|1/2| 0 |1/2| 0 | 0 |
| 0 |1/2| 0 | 0 |1/2|
| 0 | 0 | 0 | 1 | 0 |
| 0 | 0 | 0 | 0 | 1 |

To see if matrix has absorbing state and can be written in canonical form:
1. check matrix is a square matrix
2. check that at least one row has all entries, except the one on the diagonal as 0s. The entry on the diagonal
   must be a 1.
3. check if matrix is irreducible. If there are $m$ states then $I + P^1 + P^2 + ... + P^m$ must have all entries    as none zero

In [2]:
def is_absorbing(mat):
    is_square = False
    is_absorbing = False
    is_irrducible = False
    m = mat.shape
    sum_P = np.zeros((m[0],m[0]))
    if not(m[0] == m[1]):
        return False
    
    for i in range(m[0]):
        if (mat[i,i] == 1) and (np.round(np.sum(mat[i,:]),1) == 1):
            is_absorbing = True
            break
    if not(is_absorbing):
        return False
    
    init_test = np.zeros((6))
    test_irreducible = np.zeros(6)
    for i in range (6):
        init_test[i] = 1
        x = np.dot(init_test, transition_matrix_drunkard)
        y = np.nonzero(x)[1]
        for j in range(len(y.tolist()[0])):
            a = y.tolist()[0][j]
            if test_irreducible[a] == 0:
                test_irreducible[a] = 1
        init_test[i] = 0

    if np.all(test_irreducible):
        return True
    else:
        return False


In [3]:
def convert_canonical_form(mat):
    if not(is_absorbing(mat)):
        return [],0,[]
    
    m = mat.shape[0]
    list_ones = []
    for i in range(m):
        for j in range(m):
            if((mat[i,j] == 1) and (i == j)):
                list_ones.append([i,j])
    r = len(list_ones)
    t = m-r
    Q = mat
    for i in reversed(range(r)):
        Q = np.delete(Q,list_ones[i][0],1)
    for i in reversed(range(r)):
        Q = np.delete(Q,list_ones[i][1],0)
        
    for i in range(r):
        temp_1 = mat[:,list_ones[i][1]]
        if i == 0:
            R = temp_1
        else:
            R = np.concatenate((R,temp_1),1)
    del temp_1
    
    for i in reversed(range(r)):
        R = np.delete(R,list_ones[i][0],0)

    return Q, t, R

In [4]:
#Using a transition matrix to test
transition_matrix_drunkard = np.matrix([[ 1 , 0 , 0 , 0 , 0 , 0 ],
                                        [1/2, 0 ,1/2, 0 , 0 , 0 ],
                                        [ 0 ,1/2, 0 ,1/2, 0 , 0 ],
                                        [ 0 , 0 ,1/2, 0 ,1/2, 0 ],
                                        [ 0 , 0 , 0 ,1/2, 0 ,1/2],
                                        [ 0 , 0 , 0 , 0 , 0 , 1 ]])

In [5]:
Q,t,R = convert_canonical_form(transition_matrix_drunkard)
print(Q, t, R)

[[ 0.   0.5  0.   0. ]
 [ 0.5  0.   0.5  0. ]
 [ 0.   0.5  0.   0.5]
 [ 0.   0.   0.5  0. ]] 4 [[ 0.5  0. ]
 [ 0.   0. ]
 [ 0.   0. ]
 [ 0.   0.5]]


$N$ is the fundamnetal matrix of $P$ (Original transition matrix in canonical form). $N=(I-Q)^{-1}$. Each entry in $N$, $n_{i,j}$, is the expected number of times the chain is in state $j$ if it started in state $i$.

In [6]:
I = np.identity(Q.shape[0])
N = np.linalg.inv(I-Q)
N

matrix([[ 1.6,  1.2,  0.8,  0.4],
        [ 1.2,  2.4,  1.6,  0.8],
        [ 0.8,  1.6,  2.4,  1.2],
        [ 0.4,  0.8,  1.2,  1.6]])